In [61]:
import json 
import pandas as pd 

with open('regmap_IVM6311.json' , 'r') as file :
    RegData = json.load(file)

RegData = pd.DataFrame(RegData.get('RegMap'))

In [62]:
# clean the description of the field in the dataframe 
RegData['Option'] = RegData['Option'].apply(lambda x :str(x).replace('\t', ''))

In [63]:
# Replace the white sapce in the register name 
RegData['REGNAME'] = RegData['REGNAME'].apply(lambda x :str(x).replace(' ', '_').replace('-',''))

In [64]:
# register filed clean 

RegData['Name'] = RegData['Name'].apply( lambda x : x.replace('[', '').replace(':', '_').replace(']', ''))

In [65]:
RegData['REGNAME'].unique() # register names in the Dataframe 

array(['System_CTRL', 'Software_reset', 'Interrupt_management',
       'Interrupt_status_register_1', 'Interrupt_mask_register_1',
       'Interrupt_status_register_2', 'Interrupt_mask_register_2',
       'Status_register_1', 'Status_register_2', 'Status_register_3',
       'Status_register_4', 'Status_register_5', 'Status_register_6',
       'Enables_settings_1', 'Input_selection', 'SEQ_settings_1',
       'SEQ_settings_2', 'SEQ_settings_3', 'SEQ_settings_4',
       'SEQ_settings_5', 'SEQ_status_register', 'NG_setting',
       'AUTO_mode_setting', 'SEQ_low_power_mode', 'PDM_low_freq_reg',
       'ANA_SPARE_Register', 'TDM_apply_configuration', 'TDM_settings_1',
       'TDM_settings_2', 'TDM_settings_3', 'TDM_settings_4',
       'Clock_monitor_settings_1', 'Clock_monitor_settings_2',
       'TIF_register', 'TIF_Spare_Register', 'NGIF_settings_1',
       'NGIF_settings_2', 'NGIF_settings_3', 'SDM_Register_1',
       'SDM_Register_2', 'SDM_Register_3', 'DWA_Register_1',
       'DWA_Regis

In [66]:
# sample testing of the register fields 
RegData[RegData['REGNAME'] == RegData['REGNAME'].unique()[5]]


,PAGE,REG,REGNAME,POS,Name,Length,Option,Attribute,Match
25,0,05,Interrupt_status_register_2,6,fir4_clip_fault,1,Clip from FIR 4,I,
26,0,05,Interrupt_status_register_2,5,fir3_clip_fault,1,Clip from FIR 3,I,
27,0,05,Interrupt_status_register_2,4,fir2_clip_fault,1,Clip from FIR 2,I,
28,0,05,Interrupt_status_register_2,3,fir1_clip_fault,1,Clip from FIR 1,I,
29,0,05,Interrupt_status_register_2,2,bst_ocp_fault,1,BST OCP fault,I,
30,0,05,Interrupt_status_register_2,1,bst_bias_ovp_fault,1,Over-voltage from BST,I,
31,0,05,Interrupt_status_register_2,0,bst_bso_ovp_fault,1,Over-voltage from BST,I,


In [67]:
hex(int('a',16))

'0xa'

In [68]:
class Name :
    
    def __init__(self) -> None:
        self.address = 0x00
        self.page = 0x0
        
    
    @property
    def x(self):
        return self._value
        
    @x.setter
    def x(self,value):
        self._value = value 

In [73]:
# filter the dataframe with the register name 
registers = []
for register_name in RegData['REGNAME'].unique():
    # filter the register fields 
    fields=[]
    register_fields_data = RegData[RegData['REGNAME'] == register_name].reset_index()
    for _,field_data in register_fields_data.iterrows():
        reg_addr = field_data.get('REG')
        page_no = field_data.get('PAGE')
        # print(reg_addr)
        #make the class methods setter and getter pattren 
        fields.append(
            f'''
    @property
    def {field_data.get('Name').strip()}(self):
        self.{field_data.get('Name').strip()}_pos   = {field_data.get('POS')}
        self.{field_data.get('Name').strip()}_len   = {field_data.get('Length')}
        return self.{field_data.get('Name').strip()}_value
        
    @{field_data.get('Name').strip()}.setter
    def {field_data.get('Name').strip()}(self,value):
        self.{field_data.get('Name').strip()}_value = value 
        regvalue = self.{field_data.get('Name').strip()}
        # perform operation with reg value 
            '''
        )
    # class pattren 
    registers.append(
        f'''
class {register_name} :
    
    def __init__(self) -> None:
        self.address = {hex(int(reg_addr,16))}
        self.page = {page_no}
        '''+''.join(fields)
    )

print(registers)
with open('IVM6311.py', 'w') as file :
    file.writelines(' '.join(registers))

['\nclass System_CTRL :\n    \n    def __init__(self) -> None:\n        self.address = 0x0\n        self.page = 0\n        \n    @property\n    def fro_sel_0p8125mhz(self):\n        self.fro_sel_0p8125mhz_pos   = 3\n        self.fro_sel_0p8125mhz_len   = 1\n        return self.fro_sel_0p8125mhz_value\n        \n    @fro_sel_0p8125mhz.setter\n    def fro_sel_0p8125mhz(self,value):\n        self.fro_sel_0p8125mhz_value = value \n        regvalue = self.fro_sel_0p8125mhz\n        # perform operation with reg value \n            \n    @property\n    def fro_sel_1p625mhz(self):\n        self.fro_sel_1p625mhz_pos   = 2\n        self.fro_sel_1p625mhz_len   = 1\n        return self.fro_sel_1p625mhz_value\n        \n    @fro_sel_1p625mhz.setter\n    def fro_sel_1p625mhz(self,value):\n        self.fro_sel_1p625mhz_value = value \n        regvalue = self.fro_sel_1p625mhz\n        # perform operation with reg value \n            \n    @property\n    def fro_sel(self):\n        self.fro_sel_pos   =

In [70]:
# python setter and getter class 
class C(object):
    def __init__(self):
        self._x = None

    @property
    def x(self):
        """I'm the 'x' property."""
        print("getter of x called")
        return self._x

    @x.setter
    def x(self, value):
        print("setter of x called")
        self._x = value
        print(self._x)

    @x.deleter
    def x(self):
        print("deleter of x called")
        del self._x


c = C()
c.x = 'foo'  # setter called

setter of x called
foo


In [74]:
from IVM6311 import System_CTRL
cls = System_CTRL()
cls.fro_sel_0p8125mhz = 'hi'
cls.fro_sel_0p8125mhz

'Hi'